In [41]:
import json
import sys

import ipywidgets as widgets
import requests
from IPython.display import display


class HepWrapper:

    entries_limit = 1000

    @staticmethod
    @out1.capture()
    def getHepIDfromDoi(doi_string: str) -> str:
        response = requests.get(f"https://inspirehep.net/api/doi/{doi_string}")
        hit = response.json()
        # print(hit)
        return str(hit["metadata"]["control_number"])

    @staticmethod
    @out1.capture()
    def getPublicationDictFromHepID(hepID: str) -> str:
        response = requests.get(f"https://inspirehep.net/api/literature/{hepID}")
        hit = response.json()
        return hit

    @staticmethod
    @out1.capture()
    def getPublicationTitleFromPublicationDict(pubDict: str) -> str:
        return pubDict["metadata"]["titles"][0]["title"]

    @staticmethod
    @out1.capture()
    def getCitationsDictFromHepID(hepID: str) -> dict:

        entries_limit = 1000
        response = requests.get(
            f"https://inspirehep.net/api/literature?q=refersto:recid:{hepID}&size={entries_limit}"
        )
        print("parsing data..")
        hits = response.json()["hits"]["hits"]
        citations_in_year = {}

        for hit in hits:
            datestring = str(hit["created"])
            year = datestring.split("-")[0]

            if year not in citations_in_year:
                citations_in_year[year] = 1
            else:
                citations_in_year[year] = citations_in_year[year] + 1

        return citations_in_year


txtDoiInput = widgets.Text(
    value="10.1103/PhysRevLett.19.1264",
    placeholder="Type something",
    description="DOI code:",
    disabled=False,
)

btnLoadDoi = widgets.Button(description="Load DOI", button_style='primary')
btnClearOutput = widgets.Button(description="Clear output", button_style='danger')
out1 = widgets.Output(layout={"border": "1px solid black"})
link = widgets.HTML(None,)
doilink = widgets.HTML(None,)


display(txtDoiInput)
display(btnLoadDoi)
display(btnClearOutput)
display(link)
display(doilink)
display(out1)

@out1.capture()
def disableControls():
    btnLoadDoi.disabled = True
    btnClearOutput.disabled = True
    btnLoadDoi.description = "..work in progress.."


@out1.capture()
def enableControls():
    btnLoadDoi.description = "Load DOI"
    btnLoadDoi.disabled = False
    btnClearOutput.disabled = False
    
    
@out1.capture()
def on_btnLoadDoi_clicked(b):
    disableControls()
    hw = HepWrapper()
    identifier_value = hw.getHepIDfromDoi(txtDoiInput.value)    
    print(f"hepID translated: {identifier_value}")
    
    link.value=f"<a style=\"color:blue;\" href=https://inspirehep.net/literature/{identifier_value} target='_blank'>link to inspire-HEP publication page</a>"
    doilink.value=f"<a style=\"color:blue;\" href=https://doi.org/{txtDoiInput.value} target='_blank'>DOI link</a>"
                
    pub_dict = hw.getPublicationDictFromHepID(identifier_value)
    pub_title = hw.getPublicationTitleFromPublicationDict(pub_dict)
    
    if pub_title == None:
        print(f"Publication title not found. Check if DOI is valid.")
        enableControls()
        return
    
    print(f"publication title: {pub_title}")    
    print(f"Getting publication metadata...")
    cits_dict = hw.getCitationsDictFromHepID(identifier_value)
    print(f"...ok")
    
    text = ""
    y = 0
    first_year = sorted(cits_dict)[0]
    print(f"first_year: {first_year}")
    print(f"total citations: {len(cits_dict)}")
    
    for year in sorted(cits_dict):
        text = text + f"rel_year: {int(year)-int(first_year)} year: {year} citations_in_year: {cits_dict[year]}\n"
        
    print(text)
    enableControls()
    

    
@out1.capture()
def on_btnClearOutput_clicked(b):
    out1.clear_output()
    link.value=""
    doilink.value=""
        

btnLoadDoi.on_click(on_btnLoadDoi_clicked)
btnClearOutput.on_click(on_btnClearOutput_clicked)




Text(value='10.1103/PhysRevLett.19.1264', description='DOI code:', placeholder='Type something')

Button(button_style='primary', description='Load DOI', style=ButtonStyle())

Button(button_style='danger', description='Clear output', style=ButtonStyle())

HTML(value='')

HTML(value='')

Output(layout=Layout(border='1px solid black'))